In [48]:
import numpy as np
import pandas as pd
# Paquetes

# Gurobi Python Interface
from gurobipy import*

In [49]:
# Data
scenarios = [1,2,3]
hrs_week = 168
probs = {1:0.2,2:0.6,3:0.2}
demands = {1:110,2:100,3:80}
prices = {1:50,2:46,3:44}

In [50]:
# Create Model
model = Model('example 2.6')

In [51]:
# Variables
# Admission variable xi
Pc = model.addVar(name = "Pc",lb=0,ub=90,vtype=GRB.CONTINUOUS)
# Qualification indicator
Pi = model.addVars(scenarios,name="Pi",lb=0,vtype=GRB.CONTINUOUS)

# Model update
model.update()

In [52]:
# Objective
obj = hrs_week*(45*Pc) + hrs_week*quicksum(probs[scenario]*prices[scenario]*Pi[scenario] for scenario in scenarios)
model.setObjective(obj, GRB.MINIMIZE) # maximize profit

In [53]:
# Constraints
constraints = model.addConstrs(Pc + Pi[scenario] >= demands[scenario] for scenario in scenarios)

In [54]:
# Update model
model.update()

# Proceed to optimize
model.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3 rows, 4 columns and 6 nonzeros
Model fingerprint: 0x3193078e
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+03, 8e+03]
  Bounds range     [9e+01, 9e+01]
  RHS range        [8e+01, 1e+02]
Presolve removed 3 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.4793600e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  7.479360000e+05


In [55]:
# Visualization
rows = scenarios.copy() + ["contract"]
columns = ["Power bought"]
Pi_table = pd.DataFrame(columns=columns, index=rows, data=0.0)

for scenario in Pi.keys():
    if (abs(Pi[scenario].x) > 1e-6):
        Pi_table.loc[scenario, "Power bought"] = np.round(Pi[scenario].x, 1)
Pi_table.loc["contract", "Power bought"] = np.round(Pc.x, 1)
Pi_table

,Power bought
1,30.0
2,20.0
3,0.0
contract,80.0
